**Классификация изображений с помощью сверточных нейронных сетей**

В данном задании Вам необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6).
Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом *None*.
Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%
Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [1]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops

In [2]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [3]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data(
    label_mode='fine'
)

In [4]:
# Преобразуйте метки классов в one_hot формат
y_train = tf.keras.utils.to_categorical(y_train, CLASSES)
y_val = tf.keras.utils.to_categorical(y_val, CLASSES)

In [5]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [6]:
# Задайте архитектуру модели
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=[32,32,3]),
    tf.keras.layers.Conv2D(24, 4, padding='same', strides=2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPool2D(4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(CLASSES),
    tf.keras.layers.Activation('sigmoid')
])

In [7]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")

Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
FLOPs: 0.6914e6


In [8]:
# вывод краткой информации о модели
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 16, 16, 24)        1176      
                                                                 
 batch_normalization (BatchN  (None, 16, 16, 24)       96        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 16, 16, 24)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 4, 4, 24)         0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 384)               0         
                                                                 
 dense (Dense)               (None, 100)               3

In [9]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.5)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [10]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=128
)

Epoch 1/128
391/391 [==============================] - 8s 18ms/step - loss: 3.7025 - accuracy: 0.1667 - val_loss: 3.3308 - val_accuracy: 0.2254
Epoch 2/128
391/391 [==============================] - 7s 17ms/step - loss: 3.0144 - accuracy: 0.2753 - val_loss: 3.0347 - val_accuracy: 0.2767
Epoch 3/128
391/391 [==============================] - 7s 18ms/step - loss: 2.7920 - accuracy: 0.3190 - val_loss: 2.9337 - val_accuracy: 0.2913
Epoch 4/128
391/391 [==============================] - 7s 17ms/step - loss: 2.6638 - accuracy: 0.3467 - val_loss: 2.8083 - val_accuracy: 0.3276
Epoch 5/128
391/391 [==============================] - 7s 17ms/step - loss: 2.5679 - accuracy: 0.3649 - val_loss: 2.8312 - val_accuracy: 0.3173
Epoch 6/128
391/391 [==============================] - 7s 17ms/step - loss: 2.4935 - accuracy: 0.3822 - val_loss: 2.7970 - val_accuracy: 0.3256
Epoch 7/128
391/391 [==============================] - 7s 17ms/step - loss: 2.4375 - accuracy: 0.3961 - val_loss: 2.7411 - val_accuracy:

391/391 [==============================] - 7s 17ms/step - loss: 2.2368 - accuracy: 0.4395 - val_loss: 2.6619 - val_accuracy: 0.3615
Epoch 58/128
391/391 [==============================] - 7s 17ms/step - loss: 2.2384 - accuracy: 0.4373 - val_loss: 2.6618 - val_accuracy: 0.3617
Epoch 59/128
391/391 [==============================] - 7s 18ms/step - loss: 2.2365 - accuracy: 0.4396 - val_loss: 2.6616 - val_accuracy: 0.3619
Epoch 60/128
391/391 [==============================] - 7s 17ms/step - loss: 2.2392 - accuracy: 0.4386 - val_loss: 2.6613 - val_accuracy: 0.3623
Epoch 61/128
391/391 [==============================] - 7s 17ms/step - loss: 2.2358 - accuracy: 0.4404 - val_loss: 2.6623 - val_accuracy: 0.3614
Epoch 62/128
391/391 [==============================] - 7s 17ms/step - loss: 2.2365 - accuracy: 0.4393 - val_loss: 2.6615 - val_accuracy: 0.3617
Epoch 63/128
391/391 [==============================] - 7s 17ms/step - loss: 2.2378 - accuracy: 0.4382 - val_loss: 2.6619 - val_accuracy: 0.361

391/391 [==============================] - 7s 17ms/step - loss: 2.2350 - accuracy: 0.4398 - val_loss: 2.6627 - val_accuracy: 0.3619
Epoch 124/128
391/391 [==============================] - 7s 18ms/step - loss: 2.2375 - accuracy: 0.4397 - val_loss: 2.6612 - val_accuracy: 0.3617
Epoch 125/128
391/391 [==============================] - 7s 17ms/step - loss: 2.2377 - accuracy: 0.4378 - val_loss: 2.6624 - val_accuracy: 0.3615
Epoch 126/128
391/391 [==============================] - 7s 18ms/step - loss: 2.2364 - accuracy: 0.4393 - val_loss: 2.6616 - val_accuracy: 0.3614
Epoch 127/128
391/391 [==============================] - 7s 17ms/step - loss: 2.2374 - accuracy: 0.4395 - val_loss: 2.6614 - val_accuracy: 0.3614
Epoch 128/128
391/391 [==============================] - 7s 18ms/step - loss: 2.2375 - accuracy: 0.4388 - val_loss: 2.6618 - val_accuracy: 0.3624


In [11]:
scores = model.evaluate(X_val, y_val, verbose=0) 
print(f"Model accuracy: {scores[1]:.4f}")

Model accuracy: 0.3624


In [12]:
model.save("cifar-100.hdf5")